In [2]:
import numpy as np
import pandas as pd
import glob
import os
from pathlib import Path
from itertools import product
import regex as re

In [4]:

def compute_lps(pattern):
    # Longest Proper Prefix that is suffix array
    lps = [0] * len(pattern)

    prefi = 0
    for i in range(1, len(pattern)):
        
        while prefi and pattern[i] != pattern[prefi]:
            prefi = lps[prefi - 1]

        if pattern[prefi] == pattern[i]:
            prefi += 1
            lps[i] = prefi


    return lps



def kmp(pattern, text):
    match_indices = []
    count = 0
    pattern_lps = compute_lps(pattern)

    patterni = 0
    for i, ch in enumerate(text):
        
        while patterni and pattern[patterni] != ch:
            patterni = pattern_lps[patterni - 1]

        if pattern[patterni] == ch:
            if patterni == len(pattern) - 1:
                match_indices.append(i - patterni)
                count += 1
                patterni = pattern_lps[patterni]
            
            else:
                patterni += 1

    return match_indices, count


In [5]:
def remove_impossible_patterns(pattern_list):

    patterns_cleaned = list(pattern_list)

    for sub in list(patterns_cleaned):
        if 'RR' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'LL' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'JJ' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'rr' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'll' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'jj' in sub:
            patterns_cleaned.remove(sub)
            continue

    return patterns_cleaned

In [134]:

p3 = [''.join(item) for item in product('RLJ', 'RrLlJj', 'RrLlJj')]
p3_clean = remove_impossible_patterns(p3)


list_patterns_set = [[], [l for l in 'RrLlJj'], [''.join(item) for item in product('RLJ', 'RrLlJj')], p3_clean]
list_best_freq= [[], [], [], []]


In [6]:

df = pd.read_csv('/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data/keylogs/ground_truth/2022-11-25-12_13_37.csv')
df_trim = df[(df['KEY'] == 'Key.left') | (df['KEY'] == 'Key.right') | (df['KEY'] == 'Key.space')]

#R -> Down, r -> Up
df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'R'
df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'UP'), 'action'] = 'r'

df_trim.loc[(df_trim['KEY'] == 'Key.left') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'L'
df_trim.loc[(df_trim['KEY'] == 'Key.left') & (df_trim['STATUS'] == 'UP'), 'action'] = 'l'

df_trim.loc[(df_trim['KEY'] == 'Key.space') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'J'
df_trim.loc[(df_trim['KEY'] == 'Key.space') & (df_trim['STATUS'] == 'UP'), 'action'] = 'j'
df_trim

/var/folders/0y/g4f502v57jd_s8zx0025dl3w0000gn/T/ipykernel_44530/2123146158.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'R'


,FRAME,KEY,STATUS,action
4,186,Key.right,DOWN,R
6,302,Key.space,DOWN,J
7,311,Key.space,UP,j
8,330,Key.right,UP,r
10,353,Key.right,DOWN,R
...,...,...,...,...
6729,130284,Key.left,UP,l
6730,130295,Key.right,DOWN,R
6731,130323,Key.right,UP,r
6732,130323,Key.space,DOWN,J


In [119]:
candidates = dict()
best_timeframe_size = 0

for second_tick in np.linspace(3, 10, 15): #3., 3.5, 4, ..., 9.5, 10.
    n_frame = second_tick*60
    slice_size = int(df_trim.FRAME.max()/n_frame)


    freq = pd.DataFrame(index=list_patterns_set[3], columns=['count', 'frequency'])
    freq['count'] = 0
    freq['frequency'] = 0

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]

        str = slice.action.str.cat()

        for item in freq.index:
            freq.loc[item, 'count'] += kmp(item, str)[1]

    n_pattern = freq['count'].sum()

    freq = freq.loc[freq['count'] > 0]    

    for item in freq.index:
        if n_pattern > 0:
            freq.loc[item, 'frequency'] = freq.loc[item, 'count']/n_pattern * 100
        else:
            freq.loc[item, 'frequency'] = 0

    candidates[second_tick] = freq.sort_values(by=['count'], ascending=False).head(15)

    print(second_tick, n_pattern)

list_patterns_set.append([''.join(item) for item in product(freq.index, 'RrLlJj')])
list_best_freq.append(candidates)
print(len(freq.index))
candidates

3.0 1833
3.5 1903
4.0 1960
4.5 1997
5.0 2043
5.5 2080
6.0 2121
6.5 2122
7.0 2154
7.5 2163
8.0 2175
8.5 2186
9.0 2206
9.5 2219
10.0 2223
51


{3.0:      count  frequency
 JjJ    371  20.240044
 Jjr    211  11.511184
 RJj    202  11.020185
 RrR    117   6.382979
 Jjl     88   4.800873
 LlL     77   4.200764
 LlR     77   4.200764
 LJj     75   4.091653
 RrJ     60   3.273322
 JjR     59   3.218767
 JRj     54   2.945990
 Jrj     50   2.727769
 Rjr     40   2.182215
 RrL     40   2.182215
 Jlj     38   2.073104,
 3.5:      count  frequency
 JjJ    398  20.914346
 Jjr    224  11.770888
 RJj    216  11.350499
 RrR    114   5.990541
 LlR     82   4.308986
 Jjl     80   4.203889
 LJj     79   4.151340
 LlL     75   3.941146
 RrJ     70   3.678403
 JjR     61   3.205465
 JRj     52   2.732528
 Jrj     50   2.627430
 RrL     45   2.364687
 Rjr     39   2.049396
 Jlj     36   1.891750,
 4.0:      count  frequency
 JjJ    420  21.428571
 Jjr    223  11.377551
 RJj    206  10.510204
 RrR    125   6.377551
 Jjl     93   4.744898
 LlR     84   4.285714
 LJj     80   4.081633
 LlL     75   3.826531
 JjR     64   3.265306
 RrJ     59   3.0

In [120]:
candidates = dict()
best_timeframe_size = 0

for second_tick in np.linspace(3, 10, 15): #3., 3.5, 4, ..., 9.5, 10.
    n_frame = second_tick*60
    slice_size = int(df_trim.FRAME.max()/n_frame)


    freq = pd.DataFrame(index=list_patterns_set[4], columns=['count', 'frequency'])
    freq['count'] = 0
    freq['frequency'] = 0

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]

        str = slice.action.str.cat()

        for item in freq.index:
            freq.loc[item, 'count'] += kmp(item, str)[1]

    n_pattern = freq['count'].sum()

    freq = freq.loc[freq['count'] > 0]    

    for item in freq.index:
        if n_pattern > 0:
            freq.loc[item, 'frequency'] = freq.loc[item, 'count']/n_pattern * 100
        else:
            freq.loc[item, 'frequency'] = 0

    candidates[second_tick] = freq.sort_values(by=['count'], ascending=False).head(15)

    print(second_tick, n_pattern)

list_patterns_set.append([''.join(item) for item in product(freq.index, 'RrLlJj')])
list_best_freq.append(candidates)
print(len(freq.index))
candidates

3.0 1554
3.5 1650
4.0 1761
4.5 1791
5.0 1859
5.5 1914
6.0 1966
6.5 1969
7.0 2000
7.5 2044
8.0 2061
8.5 2079
9.0 2091
9.5 2111
10.0 2123
117


{3.0:       count  frequency
 JjJj    288  18.532819
 RJjr    104   6.692407
 JjrL     83   5.341055
 JjrR     69   4.440154
 RrRr     65   4.182754
 LlLl     64   4.118404
 LJjl     63   4.054054
 RJjJ     57   3.667954
 JjlR     42   2.702703
 LlRJ     36   2.316602
 RrJj     34   2.187902
 JRjr     33   2.123552
 JjRr     31   1.994852
 LJlj     30   1.930502
 RrRJ     29   1.866152,
 3.5:       count  frequency
 JjJj    323  19.575758
 RJjr    111   6.727273
 JjrL     88   5.333333
 JjrR     71   4.303030
 RrRr     68   4.121212
 RJjJ     66   4.000000
 LlLl     61   3.696970
 LJjl     61   3.696970
 LlRJ     42   2.545455
 JjlR     42   2.545455
 JjRr     35   2.121212
 RrJj     34   2.060606
 JRjr     33   2.000000
 LlRr     32   1.939394
 LJlj     30   1.818182,
 4.0:       count  frequency
 JjJj    360  20.442930
 RJjr    107   6.076093
 JjrL     92   5.224304
 JjrR     80   4.542873
 RrRr     73   4.145372
 RJjJ     71   4.031800
 LJjl     68   3.861442
 LlLl     63   3.577513

In [121]:
candidates = dict()
best_timeframe_size = 0

for second_tick in np.linspace(3, 10, 15): #3., 3.5, 4, ..., 9.5, 10.
    n_frame = second_tick*60
    slice_size = int(df_trim.FRAME.max()/n_frame)


    freq = pd.DataFrame(index=list_patterns_set[5], columns=['count', 'frequency'])
    freq['count'] = 0
    freq['frequency'] = 0

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]

        str = slice.action.str.cat()

        for item in freq.index:
            freq.loc[item, 'count'] += kmp(item, str)[1]

    n_pattern = freq['count'].sum()

    freq = freq.loc[freq['count'] > 0]    

    for item in freq.index:
        if n_pattern > 0:
            freq.loc[item, 'frequency'] = freq.loc[item, 'count']/n_pattern * 100
        else:
            freq.loc[item, 'frequency'] = 0

    candidates[second_tick] = freq.sort_values(by=['count'], ascending=False).head(15)

    print(second_tick, n_pattern)

list_patterns_set.append([''.join(item) for item in product(freq.index, 'RrLlJj')])
list_best_freq.append(candidates)
print(len(freq.index))
candidates

3.0 1259
3.5 1383
4.0 1519
4.5 1588
5.0 1657
5.5 1728
6.0 1787
6.5 1821
7.0 1845
7.5 1897
8.0 1924
8.5 1943
9.0 1968
9.5 1991
10.0 2012
219


{3.0:        count  frequency
 JjJjJ    145  11.517077
 JjJjr     44   3.494837
 JjrRJ     42   3.335981
 RJjrR     37   2.938840
 LlLlL     36   2.859412
 RJjJj     36   2.859412
 JjrLl     34   2.700556
 RJjrL     33   2.621128
 RrRrR     32   2.541700
 LJjlR     31   2.462272
 JjrLJ     30   2.382844
 LlRJj     28   2.223987
 JjrRr     20   1.588562
 RrRJj     19   1.509134
 JjlRJ     18   1.429706,
 3.5:        count  frequency
 JjJjJ    184  13.304411
 JjJjr     52   3.759942
 RJjJj     47   3.398409
 RJjrR     41   2.964570
 JjrRJ     41   2.964570
 JjrLl     39   2.819957
 RJjrL     38   2.747650
 LlLlL     34   2.458424
 RrRrR     34   2.458424
 LJjlR     33   2.386117
 JjrLJ     32   2.313811
 LlRJj     31   2.241504
 JjrRr     23   1.663051
 JjlRJ     21   1.518438
 RrRJj     17   1.229212,
 4.0:        count  frequency
 JjJjJ    200  13.166557
 JjJjr     59   3.884134
 RJjJj     57   3.752469
 JjrRJ     46   3.028308
 JjrLl     42   2.764977
 LlLlL     42   2.764977
 RJjrR  

In [122]:
candidates = dict()
best_timeframe_size = 0

for second_tick in np.linspace(3, 10, 15): #3., 3.5, 4, ..., 9.5, 10.
    n_frame = second_tick*60
    slice_size = int(df_trim.FRAME.max()/n_frame)


    freq = pd.DataFrame(index=list_patterns_set[6], columns=['count', 'frequency'])
    freq['count'] = 0
    freq['frequency'] = 0

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]

        str = slice.action.str.cat()

        for item in freq.index:
            freq.loc[item, 'count'] += kmp(item, str)[1]

    n_pattern = freq['count'].sum()

    freq = freq.loc[freq['count'] > 0]    

    for item in freq.index:
        if n_pattern > 0:
            freq.loc[item, 'frequency'] = freq.loc[item, 'count']/n_pattern * 100
        else:
            freq.loc[item, 'frequency'] = 0

    candidates[second_tick] = freq.sort_values(by=['count'], ascending=False).head(15)

    print(second_tick, n_pattern)

list_patterns_set.append([''.join(item) for item in product(freq.index, 'RrLlJj')])
list_best_freq.append(candidates)
print(len(freq.index))
candidates

3.0 1018
3.5 1158
4.0 1306
4.5 1402
5.0 1498
5.5 1577
6.0 1641
6.5 1675
7.0 1720
7.5 1775
8.0 1799
8.5 1836
9.0 1862
9.5 1890
10.0 1925
361


{3.0:         count  frequency
 JjJjJj    106  10.412574
 JjrRJj     33   3.241650
 LlLlLl     33   3.241650
 JjJjrL     22   2.161100
 JjrLJj     20   1.964637
 RJjrRJ     19   1.866405
 RrRrRr     18   1.768173
 JjlRJj     17   1.669941
 JjrLlR     15   1.473477
 RJjrLl     15   1.473477
 RrRJjr     14   1.375246
 RJjrRr     14   1.375246
 LJjlRJ     12   1.178782
 RrJjJj     12   1.178782
 RJjJjr     12   1.178782,
 3.5:         count  frequency
 JjJjJj    151  13.039724
 LlLlLl     30   2.590674
 JjrRJj     28   2.417962
 RJjrRJ     22   1.899827
 JjrLJj     22   1.899827
 RrRrRr     21   1.813472
 JjJjrL     20   1.727116
 RJjrLl     20   1.727116
 JjlRJj     18   1.554404
 RJjJjJ     17   1.468048
 JjrLlR     17   1.468048
 LJjlRJ     15   1.295337
 RJjrRr     14   1.208981
 LlRJjr     14   1.208981
 RJjrLJ     13   1.122625,
 4.0:         count  frequency
 JjJjJj    173  13.246554
 LlLlLl     39   2.986217
 JjrRJj     36   2.756508
 JjJjrL     26   1.990812
 JjrLJj     25   1.91

In [123]:
candidates = dict()
best_timeframe_size = 0

for second_tick in np.linspace(3, 10, 15): #3., 3.5, 4, ..., 9.5, 10.
    n_frame = second_tick*60
    slice_size = int(df_trim.FRAME.max()/n_frame)


    freq = pd.DataFrame(index=list_patterns_set[7], columns=['count', 'frequency'])
    freq['count'] = 0
    freq['frequency'] = 0

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]

        str = slice.action.str.cat()

        for item in freq.index:
            freq.loc[item, 'count'] += kmp(item, str)[1]

    n_pattern = freq['count'].sum()

    freq = freq.loc[freq['count'] > 0]    

    for item in freq.index:
        if n_pattern > 0:
            freq.loc[item, 'frequency'] = freq.loc[item, 'count']/n_pattern * 100
        else:
            freq.loc[item, 'frequency'] = 0

    candidates[second_tick] = freq.sort_values(by=['count'], ascending=False).head(15)

    print(second_tick, n_pattern)

list_patterns_set.append([''.join(item) for item in product(freq.index, 'RrLlJj')])
list_best_freq.append(candidates)
print(len(freq.index))
candidates

3.0 805
3.5 947
4.0 1095
4.5 1197
5.0 1317
5.5 1389
6.0 1473
6.5 1527
7.0 1578
7.5 1626
8.0 1666
8.5 1707
9.0 1741
9.5 1762
10.0 1818
548


{3.0:          count  frequency
 JjJjJjJ     62   7.701863
 LlLlLlL     20   2.484472
 JjrLJjl     14   1.739130
 JjrRJjr     14   1.739130
 RJjrRJj     13   1.614907
 JjJjJjr     12   1.490683
 JjJjrLl     11   1.366460
 LJjlRJj     11   1.366460
 RrRrRrR      9   1.118012
 JjlRJjr      7   0.869565
 RrRrRJj      7   0.869565
 JjJjrRJ      7   0.869565
 RJjJjJj      7   0.869565
 JjrJRjr      7   0.869565
 JjlLJjl      6   0.745342,
 3.5:          count  frequency
 JjJjJjJ     90   9.503696
 JjJjJjr     18   1.900739
 LlLlLlL     18   1.900739
 JjrRJjr     17   1.795143
 RJjJjJj     15   1.583949
 RJjrRJj     14   1.478353
 JjrLJjl     14   1.478353
 LJjlRJj     13   1.372756
 RrRrRrR      8   0.844773
 RJjrLlR      8   0.844773
 JjJjrLl      8   0.844773
 JjlRJjr      8   0.844773
 JjrJRjr      8   0.844773
 JjrLlRJ      8   0.844773
 RJjrLJj      8   0.844773,
 4.0:          count  frequency
 JjJjJjJ    101   9.223744
 LlLlLlL     26   2.374429
 JjrRJjr     21   1.917808
 JjrLJjl   

In [124]:
candidates = dict()
best_timeframe_size = 0

for second_tick in np.linspace(3, 10, 15): #3., 3.5, 4, ..., 9.5, 10.
    n_frame = second_tick*60
    slice_size = int(df_trim.FRAME.max()/n_frame)


    freq = pd.DataFrame(index=list_patterns_set[8], columns=['count', 'frequency'])
    freq['count'] = 0
    freq['frequency'] = 0

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]

        str = slice.action.str.cat()

        for item in freq.index:
            freq.loc[item, 'count'] += kmp(item, str)[1]

    n_pattern = freq['count'].sum()

    freq = freq.loc[freq['count'] > 0]    

    for item in freq.index:
        if n_pattern > 0:
            freq.loc[item, 'frequency'] = freq.loc[item, 'count']/n_pattern * 100
        else:
            freq.loc[item, 'frequency'] = 0

    candidates[second_tick] = freq.sort_values(by=['count'], ascending=False).head(15)

    print(second_tick, n_pattern)

list_patterns_set.append([''.join(item) for item in product(freq.index, 'RrLlJj')])
list_best_freq.append(candidates)
print(len(freq.index))
candidates

3.0 610
3.5 763
4.0 911
4.5 1017
5.0 1152
5.5 1239
6.0 1300
6.5 1380
7.0 1427
7.5 1477
8.0 1521
8.5 1568
9.0 1607
9.5 1641
10.0 1719
726


{3.0:           count  frequency
 JjJjJjJj     47   7.704918
 LlLlLlLl     19   3.114754
 JjJjJjrL      8   1.311475
 JjrLJjlR      7   1.147541
 RrRrRrRr      7   1.147541
 JjJjrLlR      6   0.983607
 JjJjrRJj      6   0.983607
 RrRrRJjr      5   0.819672
 JjrRJjJj      5   0.819672
 RJjrRJjr      5   0.819672
 LJjlRJjr      5   0.819672
 JjJjlRJj      4   0.655738
 RJjrLJjl      4   0.655738
 RrRrRrJj      4   0.655738
 LJjlLJjl      4   0.655738,
 3.5:           count  frequency
 JjJjJjJj     75   9.829620
 LlLlLlLl     16   2.096986
 RJjrRJjr      8   1.048493
 JjJjJjrL      7   0.917431
 JjrLJjlR      7   0.917431
 LJjlRJjr      7   0.917431
 JjrLlRJj      6   0.786370
 RJjJjJjr      6   0.786370
 RJjrLJjl      6   0.786370
 RrRrRrRr      6   0.786370
 JjrRJjrR      5   0.655308
 RJjJjJjJ      5   0.655308
 JjRrRrRr      5   0.655308
 LJjlLJjl      5   0.655308
 JjrRrRrR      5   0.655308,
 4.0:           count  frequency
 JjJjJjJj     86   9.440176
 LlLlLlLl     25   2.744237
 Jj

In [7]:
p3 = [''.join(item) for item in product('RLJ', 'RrLlJj', 'RrLlJj')]
p3_clean = remove_impossible_patterns(p3)


list_patterns_set = [[], [l for l in 'RrLlJj'], [''.join(item) for item in product('RLJ', 'RrLlJj')], p3_clean]

In [48]:
p3 = [''.join(item) for item in product('RLJ', 'RrLlJj', 'RrLlJj')]
p3_clean = remove_impossible_patterns(p3)


list_patterns_set = [[], [l for l in 'RrLlJj'], remove_impossible_patterns([''.join(item) for item in product('RLJ', 'RrLlJj')]), p3_clean]

freqs = pd.DataFrame(columns=)
best_freq_locs = dict()

for i in range(3,11): # on fait pour des patters de longueur 3 à 10
    print('starting size', i)
    freq = pd.DataFrame(0, index=list_patterns_set[i], columns=columns)
    freq.loc[:, pd.IndexSlice[:, 'best_freq']] = False    
    best_freq_loc = dict(zip(freq.index, [[0,0]]*len(freq.index))) # va servir a stocker la fenetre pour laquelle on a la plus haute frequence pour un pattern donné ('frequency','window')

    for window in np.linspace(2, 10, 17): #2., 2.5, 3, ..., 9.5, 10. --> On teste des intervalles à la demi-seconde
        n_frame = window*60 #nb de frame a considerer = secondes*FPS
        n_slice = int(df_trim.FRAME.max()/n_frame) #nombre de période de taille "window"

        for period in range(n_slice): #pour chaque période (cad pour chaque bloc contenant n_frame)
            slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)] #on SLICE

            str = slice.action.str.cat() #on recup la chaine de caractere des actions sur laquelle on va faire la detection

            for pattern in freq.index:
                freq.loc[pattern, (window, 'count')] += kmp(pattern, str)[1] #pour chaque pattern on compte les occurences

        n_pattern = freq[window, 'count'].sum() #nb total de pattern detecté pour une window

        #freq = freq.loc[freq[window, 'count'] > 0]   #on garde que les patterns apparaissants 

        # on calcule et record si necessaire les frequences et la plus haute frequence
        if n_pattern > 0:
            for pattern in freq.index:
                freq.loc[pattern, (window, 'frequency')] = freq.loc[pattern, (window, 'count')]/n_pattern * 100
                
                if freq.loc[pattern, (window, 'frequency')] > best_freq_loc[pattern][0]:
                    freq.loc[pattern, (best_freq_loc[pattern][1], 'best_freq')] = False
                    freq.loc[pattern, (window, 'best_freq')] = True

                    best_freq_loc[pattern][0] = freq.loc[pattern, (window, 'frequency')]
                    best_freq_loc[pattern][1] = window


        #candidates[window] = freq.sort_values(by=['count'], ascending=False).head(15)

        print(window, n_pattern)

    print(len(freq))
    #on concat
    freqs = pd.concat([freqs, freq])
    best_freq_locs = best_freq_locs | best_freq_loc

    #on crée le set de patterns de taille k+1 : si un pattern de taille k n'apparait pas dans la string,
    #alors aucun pattern de taille k+1 commencant par ses k caracteres n'existe non plus
    list_patterns_set.append([''.join(pattern) for pattern in product(freq.index, 'RrLlJj')]) 

freqs


starting size 3
2.0 1572
2.5 1715
3.0 1833
3.5 1903
4.0 1960
4.5 1997
5.0 2043
5.5 2080
6.0 2121
6.5 2122
7.0 2154
7.5 2163
8.0 2175
8.5 2186
9.0 2206
9.5 2219
10.0 2223
75
starting size 4
2.0 1223
2.5 1412
3.0 1554
3.5 1650
4.0 1761
4.5 1791
5.0 1859
5.5 1914
6.0 1966
6.5 1969
7.0 2000
7.5 2044
8.0 2061
8.5 2079
9.0 2091
9.5 2111
10.0 2123
450
starting size 5


/var/folders/0y/g4f502v57jd_s8zx0025dl3w0000gn/T/ipykernel_44530/2855089245.py:53: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  freqs = pd.concat([freqs, freq])


2.0 868
2.5 1085
3.0 1260
3.5 1384
4.0 1521
4.5 1589
5.0 1657
5.5 1729
6.0 1788
6.5 1822
7.0 1846
7.5 1897
8.0 1926
8.5 1944
9.0 1970
9.5 1991
10.0 2012
2700
starting size 6


/var/folders/0y/g4f502v57jd_s8zx0025dl3w0000gn/T/ipykernel_44530/2855089245.py:53: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  freqs = pd.concat([freqs, freq])


2.0 615
2.5 840


KeyboardInterrupt: 

In [21]:
best_freq_locs

{'RrR': [21.51082450483648, 8.0],
 'RrL': [21.51082450483648, 8.0],
 'Rrl': [21.51082450483648, 8.0],
 'RrJ': [21.51082450483648, 8.0],
 'Rrj': [21.51082450483648, 8.0],
 'RLR': [21.51082450483648, 8.0],
 'RLr': [21.51082450483648, 8.0],
 'RLl': [21.51082450483648, 8.0],
 'RLJ': [21.51082450483648, 8.0],
 'RLj': [21.51082450483648, 8.0],
 'RlR': [21.51082450483648, 8.0],
 'Rlr': [21.51082450483648, 8.0],
 'RlL': [21.51082450483648, 8.0],
 'RlJ': [21.51082450483648, 8.0],
 'Rlj': [21.51082450483648, 8.0],
 'RJR': [21.51082450483648, 8.0],
 'RJr': [21.51082450483648, 8.0],
 'RJL': [21.51082450483648, 8.0],
 'RJl': [21.51082450483648, 8.0],
 'RJj': [21.51082450483648, 8.0],
 'RjR': [21.51082450483648, 8.0],
 'Rjr': [21.51082450483648, 8.0],
 'RjL': [21.51082450483648, 8.0],
 'Rjl': [21.51082450483648, 8.0],
 'RjJ': [21.51082450483648, 8.0],
 'LRr': [21.51082450483648, 8.0],
 'LRL': [21.51082450483648, 8.0],
 'LRl': [21.51082450483648, 8.0],
 'LRJ': [21.51082450483648, 8.0],
 'LRj': [21.51

In [42]:
test = pd.DataFrame(np.random.rand(75,51), index=list_patterns_set[3], columns=columns)
test.loc[:, pd.IndexSlice[:, 'best_freq']] = False

test


window (s)      2.0                           2.5                       \
stat           count frequency best_freq     count frequency best_freq   
RrR         0.532716  0.724652     False  0.205734  0.677177     False   
RrL         0.155429  0.905780     False  0.006163  0.258924     False   
Rrl         0.697857  0.555208     False  0.131229  0.257042     False   
RrJ         0.854042  0.833901     False  0.645913  0.769345     False   
Rrj         0.627589  0.194587     False  0.641063  0.733559     False   
..               ...       ...       ...       ...       ...       ...   
JjR         0.657214  0.352800     False  0.629716  0.814478     False   
Jjr         0.548137  0.504835     False  0.592647  0.027060     False   
JjL         0.348482  0.181991     False  0.323099  0.344631     False   
Jjl         0.059628  0.141724     False  0.487819  0.410962     False   
JjJ         0.680696  0.593659     False  0.313395  0.581089     False   

window (s)      3.0                           3.5   ...      8.5       9.0   \
stat           count frequency best_freq     count  ... best_freq     count   
RrR         0.613655  0.605398     False  0.626813  ...     False  0.377178   
RrL         0.759151  0.600518     False  0.687077  ...     False  0.673197   
Rrl         0.124477  0.050964     False  0.981393  ...     False  0.478230   
RrJ         0.876132  0.889787     False  0.221544  ...     False  0.784279   
Rrj         0.911786  0.670043     False  0.066815  ...     False  0.888654   
..               ...       ...       ...       ...  ...       ...       ...   
JjR         0.485389  0.876572     False  0.151186  ...     False  0.589928   
Jjr         0.328628  0.236849     False  0.326873  ...     False  0.070891   
JjL         0.652555  0.965233     False  0.909467  ...     False  0.638987   
Jjl         0.679950  0.656749     False  0.699211  ...     False  0.448622   
JjJ         0.427443  0.320604     False  0.097884  ...     False  0.978683   

window (s)                          9.5                           10.0  \
stat       frequency best_freq     count frequency best_freq     count   
RrR         0.021112     False  0.739825  0.658102     False  0.887132   
RrL         0.943954     False  0.399200  0.330410     False  0.891984   
Rrl         0.580585     False  0.363587  0.175724     False  0.375267   
RrJ         0.237961     False  0.230051  0.370313     False  0.693218   
Rrj         0.247556     False  0.488450  0.819935     False  0.745396   
..               ...       ...       ...       ...       ...       ...   
JjR         0.371174     False  0.324303  0.039246     False  0.280769   
Jjr         0.569842     False  0.153573  0.345910     False  0.061280   
JjL         0.427796     False  0.550444  0.604553     False  0.170072   
Jjl         0.726349     False  0.007967  0.078428     False  0.588141   
JjJ         0.957586     False  0.285123  0.470939     False  0.884996   

window (s)                      
stat       frequency best_freq  
RrR         0.164517     False  
RrL         0.676098     False  
Rrl         0.004170     False  
RrJ         0.752207     False  
Rrj         0.810476     False  
..               ...       ...  
JjR         0.968275     False  
Jjr         0.175119     False  
JjL         0.358054     False  
Jjl         0.123474     False  
JjJ         0.379891     False  

[75 rows x 51 columns]

In [ ]:
test.loc[test[:, 'count'] > 0.5]   #on garde que les patterns apparaissants 


window (s)      2.0                           2.5                       \
stat           count frequency best_freq     count frequency best_freq   
RrR         0.532716  0.724652     False  0.205734  0.677177     False   
Rrl         0.697857  0.555208     False  0.131229  0.257042     False   
RrJ         0.854042  0.833901     False  0.645913  0.769345     False   
Rrj         0.627589  0.194587     False  0.641063  0.733559     False   
RLR         0.984064  0.531006     False  0.470420  0.890412     False   
RLl         0.709289  0.305428     False  0.934860  0.849305     False   
RlR         0.931391  0.782661     False  0.685914  0.054074     False   
RlL         0.635471  0.315524     False  0.131157  0.983504     False   
Rlj         0.691390  0.710851     False  0.235231  0.786932     False   
RJR         0.883014  0.676763     False  0.335445  0.511001     False   
RJr         0.950452  0.927625     False  0.894566  0.991998     False   
Rjr         0.690308  0.945528     False  0.752784  0.728967     False   
Rjl         0.777490  0.843378     False  0.231094  0.005329     False   
RjJ         0.998873  0.456549     False  0.231232  0.980767     False   
LRr         0.885475  0.427785     False  0.701935  0.648988     False   
LRL         0.798613  0.838588     False  0.975274  0.153449     False   
LRl         0.840460  0.286449     False  0.417870  0.085173     False   
LRJ         0.700073  0.344011     False  0.538555  0.794358     False   
LRj         0.884582  0.545423     False  0.623698  0.763548     False   
LrR         0.598532  0.786823     False  0.815687  0.668641     False   
LrL         0.635377  0.459982     False  0.016687  0.046313     False   
Llr         0.647640  0.546244     False  0.856354  0.608924     False   
LlJ         0.996691  0.433077     False  0.944668  0.685735     False   
LJR         0.863526  0.208583     False  0.584724  0.427845     False   
LjL         0.921997  0.912912     False  0.447390  0.697887     False   
Ljl         0.765398  0.106701     False  0.070405  0.465862     False   
JRr         0.974384  0.395009     False  0.530285  0.575963     False   
JRJ         0.565702  0.688727     False  0.789058  0.645304     False   
JRj         0.662243  0.836335     False  0.045877  0.422304     False   
JrR         0.529906  0.366974     False  0.416185  0.219677     False   
JrJ         0.759172  0.020882     False  0.941086  0.962611     False   
Jrj         0.812879  0.393700     False  0.104254  0.124387     False   
JLl         0.829632  0.493947     False  0.306970  0.373940     False   
JLj         0.623299  0.571247     False  0.010083  0.344042     False   
Jlr         0.518258  0.587256     False  0.519509  0.241792     False   
JlJ         0.615920  0.749795     False  0.326790  0.276514     False   
JjR         0.657214  0.352800     False  0.629716  0.814478     False   
Jjr         0.548137  0.504835     False  0.592647  0.027060     False   
JjJ         0.680696  0.593659     False  0.313395  0.581089     False   

window (s)      3.0                           3.5   ...      8.5       9.0   \
stat           count frequency best_freq     count  ... best_freq     count   
RrR         0.613655  0.605398     False  0.626813  ...     False  0.377178   
Rrl         0.124477  0.050964     False  0.981393  ...     False  0.478230   
RrJ         0.876132  0.889787     False  0.221544  ...     False  0.784279   
Rrj         0.911786  0.670043     False  0.066815  ...     False  0.888654   
RLR         0.034402  0.310499     False  0.005845  ...     False  0.093379   
RLl         0.878277  0.370951     False  0.886755  ...     False  0.600241   
RlR         0.263946  0.501979     False  0.609728  ...     False  0.078925   
RlL         0.305307  0.788935     False  0.715438  ...     False  0.893155   
Rlj         0.953796  0.150420     False  0.803687  ...     False  0.263578   
RJR         0.395779  0.400974     False  0.374903  ...     False  0.297080   
RJr         0.942

In [3]:

df = pd.read_csv('/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data/keylogs/ground_truth/2022-11-25-12_13_37.csv')
df_trim = df[(df['KEY'] == 'Key.left') | (df['KEY'] == 'Key.right') | (df['KEY'] == 'Key.space')]

#R -> Down, r -> Up
df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'R'
df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'UP'), 'action'] = 'r'

df_trim.loc[(df_trim['KEY'] == 'Key.left') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'L'
df_trim.loc[(df_trim['KEY'] == 'Key.left') & (df_trim['STATUS'] == 'UP'), 'action'] = 'l'

df_trim.loc[(df_trim['KEY'] == 'Key.space') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'J'
df_trim.loc[(df_trim['KEY'] == 'Key.space') & (df_trim['STATUS'] == 'UP'), 'action'] = 'j'
df_trim.action.str.cat()

/var/folders/0y/g4f502v57jd_s8zx0025dl3w0000gn/T/ipykernel_51138/249272645.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'R'


'RJjrRrJRjrJRrRjrJLljLJljLlRJjJjJjJjJjJLjrJljRJjrLlRJjJjrLJjRlrRrRJjJjJjrLJljRJjrLlRJrjRJjrLlJjRJLrljLJljRrRJjrLlLJjlRLJrjlRrRJjJjJrjJRjJjrRJjrRJjJjJjJjJjrRJjrJjRrRrRrRrRrRrRJjJjJjJjJjJjrRJjJjJjJrjRrRrLlRrRrRrJjJjJjRrRrRrJjJjLlRJjrJRjrJjJjRJjrLlRJjrRJrjRrRJrjLlRJrjRJrjRJjrRrRrRJjrLJljRrJRrjRrLJjlJjRJjJjJjJjLrJjJjJjJjlRrRJjrRJjrRrRJjJjJjJjJjJjJrjLJjJjlRJjrLJljLlRJjrLJRljrRrRJjrJjRJjJjrLlRrLlRJjrLlRrLlRrRrJRjJjrLlJjRrJjLlJjLJljRrJRrjLlRrRrRrJjRrJjLlRrJjRrRrJLljRJjrRrLlRJjJjJjJjJjJjJjJjJjrLJjlRrLlJjJjJjJjJjJjJjLlJRrjRJjrJjJjJjJjRJjrLJjlJjJjJjJjRrRJjrLlLlLlRJjJjJjrLlRJjrRrJjRrRrRrJjLlJjRJrjRrRJjJjrRrRrRrLlJjRrJLjlLJjlRrLJjlRrJRjrLJjlJjLJljLlLlLlLlLlLlLlLlLlLlJjRrJLjlJLljLJjlLJjlJjRrLJljJjLlRrLJljJLRljrLlLJljLlJLjlJLjlLJjlRJjrLJjlRrJRjrLlLJljLJjlJjRrLJjlJLljRrJjRJjJjJjJjJjJjJjrLJjJjJjlRJjrRrLlLlLlRJjJjJjrJRjrRJjrRrRrLlLlRJjLrlRrLJjlRJjrRJjrRrRJjrRJrjRrLJjlRrRJjrLlRrJRjrRrRJjJjrLlRJjrJRjrRrRrRrLlRJjrLJjlRrRrLJjlRJjJjrLlRrLlRJjJjJjrRJjrLJjlRrJLjlRJjrRrJjJLjlRrRJjrRJjrLJjlRrJRjrLlJjRJjrRJrjJjL